This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [5]:
# get api key from your .env file
# import os
# from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

# load_dotenv()
API_KEY = 'yzTh15wPz51yzAbNtUzJ';#os.getenv('NASDAQ_API_KEY')

print(API_KEY)

yzTh15wPz51yzAbNtUzJ


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [6]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [7]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
stock_data_url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
start_date = '2020-01-01'
end_date = '2020-01-04'
input_params = {'start_date' : start_date, 'end_date' : end_date, 'api_key' : API_KEY}
r = requests.get(stock_data_url, params = input_params)
print(r.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-01-01', 'end_date': '2020-01-04', 'frequency': 'daily', 'data': [['2020-01-03', 112.8, 113.4, 112.0, 113.4, None, 82304.0, 9289987.0, None, None, None], ['2020-01-02', 114.0, 114.9, 112.7, 113.8, None, 143464.0, 16291666.0, None, None, None]], 'collapse': None, 'order': None}}


In [8]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
# Step 1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X

import json

stock_data_url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
start_date = '2017-01-01'
end_date = '2017-12-31'
input_params = {'start_date' : start_date, 'end_date' : end_date, 'api_key' : API_KEY}
data = requests.get(stock_data_url, params = input_params).json()

In [10]:
#Step 2: Convert the returned JSON object into a Python dictionary.
data_list = data['dataset_data']['data']; #extracting data from the dataset
column_names = data['dataset_data']['column_names'][1:];  #extracting from column_names info of that day
print("column_names:", column_names)

#creating a dictionary where key = date, value = info of that day
my_data_dict = {}
DATE_IDX = 0
for entry in data_list:
    entry_date = entry[DATE_IDX]; # a date key
    entry_data = entry[(DATE_IDX+1):] # all the day's info
    my_data_dict[entry_date] = entry_data

column_names: ['Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [11]:
#Step 3: Calculate what the highest and lowest opening prices were for the stock in this period.

#initializing the max & min values
max_open = -float("inf")
min_open = float("inf");
OPEN_IDX = 0 
for k, v in my_data_dict.items():
    if( str(v[OPEN_IDX]) != 'None'):  # ignore None value
        open_val = float(v[OPEN_IDX]) 
    if(open_val > max_open):   # update max
        max_open = open_val
    if(open_val < min_open):   # update min
        min_open = open_val
print('max open: {:6.3f}, min open: {:6.3f}'.format(max_open,min_open))

max open: 53.110, min open: 34.000


In [12]:
#Step 4: What was the largest change in any one day (based on High and Low price)?
HIGH_PRICE_IDX = 1
LOW_PRICE_IDX = 2
max_change = -float("inf")
for k, v in my_data_dict.items(): #scanning all days 
    good_val = (str(v[HIGH_PRICE_IDX]) != 'None') # ignore None value
    good_val &= (str(v[LOW_PRICE_IDX]) != 'None') # ignore None value
    good_val &= (v[HIGH_PRICE_IDX] > v[LOW_PRICE_IDX]) # data makes sense
    
    if(good_val):  
        change = v[HIGH_PRICE_IDX] - v[LOW_PRICE_IDX]
    if(change > max_change):   # update max
        max_change = change

print('max change: {:6.3f}'.format(max_change))

max change:  2.810


In [13]:
#Step 5: What was the largest change between any two days (based on Closing Price)?
CLOSE_PRICE_IDX = 3
max_change = -float("inf")

# dictionary needs sorting
# sorted makes it a tuple, sort by first element (the key) 
sorted_keys = sorted(my_data_dict.items(), key = lambda kv: kv[0])

my_data_dict_sorted = dict(sorted_keys)
first_key_sorted = sorted_keys[0][DATE_IDX]
prev_close = my_data_dict[first_key_sorted][CLOSE_PRICE_IDX] # closing price from first sorted key 

i = 0
for item in my_data_dict.items(): #scanning all items
    k = sorted_keys[i][DATE_IDX] # take the sorted key
    v = my_data_dict[k] # take the sorted value
    good_val = (str(v[CLOSE_PRICE_IDX]) != 'None') # ignore None value
    
    if(good_val):  
        change = prev_close - v[CLOSE_PRICE_IDX]
        
    if(change > max_change):   # update max
        max_change = change
      

    prev_close = v[CLOSE_PRICE_IDX] # update previous close
    i += 1
    
print('max change in closing price: {:6.3f}'.format(max_change))

max change in closing price:  2.560


In [14]:
#Step 6: What was the average daily trading volume during this year?
VOL_IDX = 5
vol_sum = 0
for k, v in my_data_dict.items(): #scanning all items
    good_val = (str(v[VOL_IDX]) != 'None') # ignore None value   
    if(good_val):  
        vol_sum += v[VOL_IDX]     

volume_avg = vol_sum / len(my_data_dict.keys())
print('The average trading volume: {:6.0f}'.format(volume_avg))

The average trading volume:  89124


In [15]:
#Step 7: (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

def find_median(array):
    median = -1
    array_sorted = sorted(array) # sort the array
    cnt = int(len(array_sorted)) 
    half = int(cnt/2)
    if((cnt % 2) == 0):
        median = (array_sorted[half] + array_sorted[half-1])/2
    else:
        median = array_sorted[half]
        
    return median

vol_list = [] # create a list of volumes
for k, v in my_data_dict.items(): #scanning all items
    good_val = (str(v[VOL_IDX]) != 'None') # ignore None value   
    if(good_val):  
        vol_list.append(v[VOL_IDX])  # add to list

print("The median trading volume is: {:6.0f}".format(find_median(vol_list)))


The median trading volume is:  76286
